In [13]:
import json
import requests
import tensorflow as tf
import base64

In [19]:
# 49,F,NAP,160,180,0,Normal,156,N,1,Flat,1

data = {
            "Age": 40,
            "Sex": "F",
            "ChestPainType": "NAP",
            "RestingBP": 160,
            "Cholesterol": 180,
            "FastingBS": 0,
            "RestingECG": "Normal",
            "MaxHR": 156,
            "ExerciseAngina": "n",
            "Oldpeak": 1,
            "ST_Slope": "Flat"
        }

In [25]:
print(data['Age'])

40


In [49]:
def prepare(data):
    # Create an Example proto from your feature dict.
    feature_spec = {
        "Age": tf.train.Feature(int64_list=tf.train.Int64List(value=[data['Age']])),
        "Sex": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(data['Sex'], "utf-8")])),
        "ChestPainType": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(data['ChestPainType'], "utf-8")])),
        "RestingBP": tf.train.Feature(int64_list=tf.train.Int64List(value=[data['RestingBP']])),
        "Cholesterol": tf.train.Feature(int64_list=tf.train.Int64List(value=[data['Cholesterol']])),
        "FastingBS": tf.train.Feature(int64_list=tf.train.Int64List(value=[data['FastingBS']])),
        "RestingECG": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(data['RestingECG'], "utf-8")])),
        "MaxHR": tf.train.Feature(int64_list=tf.train.Int64List(value=[data['MaxHR']])),
        "ExerciseAngina": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(data['ExerciseAngina'], "utf-8")])),
        "Oldpeak": tf.train.Feature(float_list=tf.train.FloatList(value=[data['Oldpeak']])),
        "ST_Slope": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(data['ST_Slope'], "utf-8")]))
    }
    
    example = tf.train.Example(features=tf.train.Features(feature=feature_spec)).SerializeToString()
    result = [{'examples': {'b64': base64.b64encode(example).decode('utf-8')}}]
    
    return json.dumps(
        {
            "signature_name":"serving_default",
            "instances": result
        }
    )

In [47]:
print(prepare(data))

{"instances": [{"b64": "CuUBCgwKA0FnZRIFGgMKASgKDwoFTWF4SFISBhoECgKcAQoTCglSZXN0aW5nQlASBhoECgKgAQoYCgpSZXN0aW5nRUNHEgoKCAoGTm9ybWFsChMKB09sZHBlYWsSCBIGCgQAAIA/CgwKA1NleBIFCgMKAUYKEgoJRmFzdGluZ0JTEgUaAwoBAAoUCghTVF9TbG9wZRIICgYKBEZsYXQKFQoLQ2hvbGVzdGVyb2wSBhoECgK0AQoYCg1DaGVzdFBhaW5UeXBlEgcKBQoDTkFQChcKDkV4ZXJjaXNlQW5naW5hEgUKAwoBbg=="}]}


In [50]:
endpoint = "https://mlops-tfx-production-1022.up.railway.app/v1/models/cc-model:predict"
 
response = requests.post(endpoint, json=prepare(data))
 
print(response.json())

{'error': 'JSON Value: "{\\"signature_name\\": \\"serving_default\\", \\"instances\\": [{\\"examples\\": {\\"b64\\": \\"CuUBChIKCUZhc3RpbmdCUxIFGgMKAQAKFAoIU1RfU2xvcGUSCAoGCgRGbGF0ChgKDUNoZXN0UGFpblR5cGUSBwoFCgNOQVAKFQoLQ2hvbGVzdGVyb2wSBhoECgK0AQoPCgVNYXhIUhIGGgQKApwBChcKDkV4ZXJjaXNlQW5naW5hEgUKAwoBbgoTCglSZXN0aW5nQlASBhoECgKgAQoTCgdPbGRwZWFrEggSBgoEAACAPwoMCgNBZ2USBRoDCgEoChgKClJlc3RpbmdFQ0cSCgoICgZOb3JtYWwKDAoDU2V4EgUKAwoBRg==\\"}}]}" Is not object'}
